In [ ]:
%%bash
mkdir data
cd data
    
wget http://biodatageeks.ii.pw.edu.pl/sequila/data/WES/NA12878.proper.wes.chr1.md.bam
wget http://biodatageeks.ii.pw.edu.pl/sequila/data/WES/Homo_sapiens_assembly18.chr1.fasta
wget http://biodatageeks.ii.pw.edu.pl/sequila/data/WES/Homo_sapiens_assembly18.chr1.fasta.fai

In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
import os
base_path = f"{os.getcwd()}/data"
bam_path = f'{base_path}/NA12878.proper.wes.chr1.md.bam'
ref_path=f'{base_path}/Homo_sapiens_assembly18.chr1.fasta'
sample_id= 'NA12878'
table_name = 'reads'
app_name = "sequila"

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.appName(f'{app_name}') \
.config('spark.driver.memory','16g') \
.getOrCreate()

In [ ]:
from pysequila import SequilaSession
ss = SequilaSession(spark)
ss.sql(f'''DROP TABLE IF EXISTS {table_name}''')
ss.sql(f'''CREATE TABLE IF NOT EXISTS {table_name} \
         USING org.biodatageeks.sequila.datasources.BAM.BAMDataSource \
         OPTIONS(path "{bam_path}")''')

In [ ]:
import pandas as pd
pd.options.display.max_columns = None
ss.sql(f"SELECT * FROM {table_name} LIMIT 5").toPandas()

In [ ]:
df = ss.sql(f"SELECT * FROM  pileup('{table_name}', '{sample_id}', '{ref_path}', true) WHERE contig='1'").toPandas()

In [ ]:
df['baf'] = df.countNonRef/df.coverage
df2 = df[(((df.baf > 0.35) & (df.baf< 0.65)) | (df.baf > 0.9)) & (df.coverage>20) & (df.coverage<300)]

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15,3]
plt.scatter(df2.pos_start, df2.baf, s=30,alpha=0.1)
plt.xlabel('Chromosome 1')
plt.ylabel('B-allele frequency')
plt.ylim(0,1)
plt.show()